In [ ]:
#!pip install ultralytics


  Using cached ultralytics-8.3.226-py3-none-any.whl.metadata (37 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached ultralytics_thop-2.0.18-py3-none-any.whl.metadata (14 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
Using cached ultralytics-8.3.226-py3-none-any.whl (1.1 MB)
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 1.5 MB/s eta 0:00:05
   ------- -------------------------------- 1.6/8.1 MB 3.4 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/8.1 MB 3.4 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/8.1 MB 3.4 MB/s eta 0:00:02
   ------------------- ---------------

In [9]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
from ultralytics import YOLO
import torch, gc

# --- Clear GPU memory before training ---
gc.collect()
torch.cuda.empty_cache()

# --- Load small YOLO model (good for 4 GB GPUs) ---
model = YOLO("yolov8n.pt")   # nano version – fast and light

# --- Train ---
model.train(
    data="C:/Users/giorg/Downloads/drone-detection-distance-template/drone-detection-distance/data/trainval_real.yaml",
    epochs=50,             # enough for small dataset
    imgsz=512,             # 512 balances detail & memory
    batch=4,               # ⬆ can try 4 or even 8 if GPU allows
    workers=0,             # avoid memory pinning issues
    cache=False,           # don’t preload dataset
    mosaic=0.0,            # reduce VRAM peaks
    mixup=0.0,
    copy_paste=0.0,
    lr0=0.001,             # learning rate
    patience=20,           # early stopping
    device=0,              # use your GPU
    name="lrdd_subset_train"
)


In [ ]:
import os
from datetime import datetime

# Base folder for YOLO project
base_dir = r"C:\Users\giorg\Downloads\drone-detection-distance-template\drone-detection-distance\data"

# List of your YAML dataset configs
experiments = [
    ("virtual-only", os.path.join(base_dir, "virtual.yaml")),
    ("real-only", os.path.join(base_dir, "real_LRDD", "real.yaml")),
    ("mixed-50-50", os.path.join(base_dir, "mixed_LRDD", "mixed.yaml")),
    # ("mixed-3-1", os.path.join(base_dir, "mixed_3to1.yaml")),  # optional
]

# YOLO base command (you can tweak epochs, imgsz, batch, etc.)
base_cmd = "yolo task=detect mode=train model=yolov8n.pt epochs=50 imgsz=640 batch=16 project=runs_yolo"

# Run sequentially
for name, yaml_path in experiments:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    run_name = f"{name}_{timestamp}"
    cmd = f'{base_cmd} data="{yaml_path}" name="{run_name}"'
    print(f"\n🚀 Starting training: {name}")
    print(f"Command: {cmd}")
    os.system(cmd)

print("\n✅ All experiments completed!")
